### Import Libraries

In [1]:
import os
import datetime

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders

import sys
from os.path import basename
from mimetypes import guess_type


from email.message import EmailMessage
from email.utils import make_msgid
import mimetypes

### Send an email

In [3]:
def report_lcms_status(LJ_instr_status, SH_instr_status):

    today = datetime.datetime.today().strftime("%B %d, %Y, %H:%M:%S")
    # strftime('%Y_%m_%d_%Hh%mm')

    msg = EmailMessage()

    mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
    mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_LJ = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_SH = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com']

    
    mail_receiver_list_US = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                             'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    
    if (LJ_instr_status == 0) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list1
        mail_subject = 'Great, QC passed for all instruments!'
        mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

    elif (LJ_instr_status == 0) & (SH_instr_status == 1):
        mail_receiver = mail_receiver_list_SH
        mail_subject = 'LJ: QC Passed; SH: QC Failed!'
        mail_content = f'Hey there, \n\nQC passed for LJ but not SH. Please doulbe check Spring House instruments!'        

    elif (LJ_instr_status == 1) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list_LJ
        mail_subject = 'LJ: QC Failed; SH: QC Passed!'
        mail_content = f'Hey there, \n\nQC passed for SH but not LJ. Please doulbe check La Jolla instruments!'   
        
    else:
        mail_receiver = mail_receiver_list_US
        mail_subject = 'OMG Super Important Message: QC failed for both LJ and SH!'
        mail_content = f'Hey there, \n\nQC failed. Please doulbe check both Spring House and La Jolla instruments!'


    mail_receiver = ', '.join(mail_receiver)


    # generic email headers
    msg['Subject'] = mail_subject

    msg['From'] = mail_sender
    msg['To'] = mail_receiver

    # set the plain text body
    msg.set_content('This is a plain text body.')

    # now create a Content-ID for the image
    # image_cid = make_msgid(domain='xyz.com')
    image_cid1 = make_msgid()
    image_cid1b = make_msgid()
    image_cid2 = make_msgid()
    image_cid2b = make_msgid()
    # if `domain` argument isn't provided, it will 
    # use your computer's name

    # set an alternative html body
    msg.add_alternative("""\
    <html>
        <body>
            <p>{date}<br><br>           
            </p>

            <p>{mail_content}<br><br>           
            </p>

            <p>La Jolla:<br>           
            </p>

            <img src="cid:{image_cid1}"><br>
            <img src="cid:{image_cid1b}"><br>

            <p>Spring House:<br>

            </p>        

            <img src="cid:{image_cid2}">
            <img src="cid:{image_cid2b}">
        </body>
    </html>
    """.format(mail_content = mail_content, date = today,\
               image_cid1=image_cid1[1:-1], image_cid1b=image_cid1b[1:-1],\
               image_cid2=image_cid2[1:-1], image_cid2b=image_cid2b[1:-1]), subtype='html')
    # image_cid looks like <long.random.number@xyz.com>
    # to use it as the img src, we don't need `<` or `>`
    # so we use [1:-1] to strip them off
    date = datetime.datetime.today().strftime("%Y_%m_%d") 
    
    fig_dir = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure'
    
    fig_name_SH = ''.join(['SH_', 'LCMS_Status_', date, '.png'])  
    
    fig_path_SH = os.path.join(fig_dir, fig_name_SH)

    fig_name_LJ = ''.join(['LJ_', 'LCMS_Status_', date, '.png'])  
    
    fig_path_LJ = os.path.join(fig_dir, fig_name_LJ)
    
#     png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_LCMS_Status_2020_08_11.png'
    legend_path_SH = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_Legend.png'

#     png_link1 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\LJ_LCMS_Status_2020_08_11.png'
    legend_path_LJ = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\LJ_Legend.png'

    # now open the image and attach it to the email
    with open(fig_path_LJ, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid1)

    with open(legend_path_LJ, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid1b)        
        
        

    # now open the image and attach it to the email
    with open(fig_path_SH, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2)

    with open(legend_path_SH, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2b)                
        
        
    #Create SMTP session for sending the mail
    try:
        server = smtplib.SMTP('smtp.na.jnj.com', 25)
        server.ehlo()
        text = msg.as_string()
        server.sendmail(mail_sender, mail_receiver, text)
        server.close()
        print(f'\nEmail sent! \n{mail_subject}')
    except:
        print('Something went wrong...')
        pass

    
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 0)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 0)


Email sent! 
OMG Super Important Message: QC failed for both LJ and SH!

Email sent! 
LJ: QC Failed; SH: QC Passed!

Email sent! 
LJ: QC Passed; SH: QC Failed!

Email sent! 
Great, QC passed for all instruments!


# Development

### Pass

In [28]:
today = datetime.datetime.today().strftime("%B %m, %Y, %H:%M:%S")
# strftime('%Y_%m_%d_%Hh%mm')

msg = EmailMessage()

mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
mail_receiver_list2 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                       'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                       'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']

instr_status = 0


if instr_status == 0:
    mail_receiver = mail_receiver_list1
    mail_subject = 'Great, QC passed for all instruments!'
    mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

else:
    mail_receiver = mail_receiver_list2
    mail_subject = 'OMG Super Important Message: failed!'
    mail_content = f'Hey there, \n\nQC failed. Please doulbe check the instruments!'


mail_receiver = ', '.join(mail_receiver)


# generic email headers
msg['Subject'] = mail_subject

msg['From'] = mail_sender
msg['To'] = mail_receiver

# set the plain text body
msg.set_content('This is a plain text body.')

# now create a Content-ID for the image
# image_cid = make_msgid(domain='xyz.com')
image_cid = make_msgid()
image_cid2 = make_msgid()
# if `domain` argument isn't provided, it will 
# use your computer's name

# set an alternative html body
msg.add_alternative("""\
<html>
    <body>
        <p>{date}<br><br>           
        </p>

        <p>{mail_content}<br><br>           
        </p>
    
        <p>La Jolla:<br>           
        </p>
        
        <img src="cid:{image_cid}"><br>
        
        <p>Spring House:<br>
           
        </p>        
        
        <img src="cid:{image_cid2}">
    </body>
</html>
""".format(mail_content = mail_content, date = today, image_cid=image_cid[1:-1], image_cid2=image_cid2[1:-1]), subtype='html')
# image_cid looks like <long.random.number@xyz.com>
# to use it as the img src, we don't need `<` or `>`
# so we use [1:-1] to strip them off


png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\SH_LCMS_Status_2020_08_07.png'

png_link = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\LJ_LCMS_Status_2020_08_07.png'

# now open the image and attach it to the email
with open(png_link, 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid)

    
# now open the image and attach it to the email
with open(png_link2, 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid2)
    

# the message is ready now
# you can write it to a file
# or send it using smtplib


#Create SMTP session for sending the mail
try:
    server = smtplib.SMTP('smtp.na.jnj.com', 25)
    server.ehlo()
    text = msg.as_string()
    server.sendmail(mail_sender, mail_receiver, text)
    server.close()
    print(f'\nEmail sent! \n{mail_subject}')
except:
    print('Something went wrong...')
    pass


Email sent! 
Great, QC passed for all instruments!


### Fail

In [27]:
today = datetime.datetime.today().strftime("%B %m, %Y, %H:%M:%S")
# strftime('%Y_%m_%d_%Hh%mm')

msg = EmailMessage()

mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
mail_receiver_list2 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                       'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                       'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']

instr_status = 1


if instr_status == 0:
    mail_receiver = mail_receiver_list1
    mail_subject = 'Great, QC passed for all instruments!'
    mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

else:
    mail_receiver = mail_receiver_list2
    mail_subject = 'OMG Super Important Message: failed!'
    mail_content = f'Hey there, \n\nQC failed. Please doulbe check the instruments!'


mail_receiver = ', '.join(mail_receiver)


# generic email headers
msg['Subject'] = mail_subject

msg['From'] = mail_sender
msg['To'] = mail_receiver

# set the plain text body
msg.set_content('This is a plain text body.')

# now create a Content-ID for the image
# image_cid = make_msgid(domain='xyz.com')
image_cid = make_msgid()
image_cid2 = make_msgid()
# if `domain` argument isn't provided, it will 
# use your computer's name

# set an alternative html body
msg.add_alternative("""\
<html>
    <body>
        <p>{date}<br><br>           
        </p>

        <p>{mail_content}<br><br>           
        </p>
    
        <p>La Jolla:<br>           
        </p>
        
        <img src="cid:{image_cid}"><br>
        
        <p>Spring House:<br>
           
        </p>        
        
        <img src="cid:{image_cid2}">
    </body>
</html>
""".format(mail_content = mail_content, date = today, image_cid=image_cid[1:-1], image_cid2=image_cid2[1:-1]), subtype='html')
# image_cid looks like <long.random.number@xyz.com>
# to use it as the img src, we don't need `<` or `>`
# so we use [1:-1] to strip them off


png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\SH_LCMS_Status_2020_08_07.png'

png_link = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\LJ_LCMS_Status_2020_08_07.png'

# now open the image and attach it to the email
with open(png_link, 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid)

    
# now open the image and attach it to the email
with open(png_link2, 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid2)
    

# the message is ready now
# you can write it to a file
# or send it using smtplib


#Create SMTP session for sending the mail
try:
    server = smtplib.SMTP('smtp.na.jnj.com', 25)
    server.ehlo()
    text = msg.as_string()
    server.sendmail(mail_sender, mail_receiver, text)
    server.close()
    print(f'\nEmail sent! \n{mail_subject}')
except:
    print('Something went wrong...')
    pass


Email sent! 
OMG Super Important Message: failed!


## Define a function

In [31]:
def report_lcms_status(instr_status):

    today = datetime.datetime.today().strftime("%B %m, %Y, %H:%M:%S")
    # strftime('%Y_%m_%d_%Hh%mm')

    msg = EmailMessage()

    mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
    mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    mail_receiver_list2 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']

    if instr_status == 0:
        mail_receiver = mail_receiver_list1
        mail_subject = 'Great, QC passed for all instruments!'
        mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

    else:
        mail_receiver = mail_receiver_list2
        mail_subject = 'OMG Super Important Message: failed!'
        mail_content = f'Hey there, \n\nQC failed. Please doulbe check the instruments!'


    mail_receiver = ', '.join(mail_receiver)


    # generic email headers
    msg['Subject'] = mail_subject

    msg['From'] = mail_sender
    msg['To'] = mail_receiver

    # set the plain text body
    msg.set_content('This is a plain text body.')

    # now create a Content-ID for the image
    # image_cid = make_msgid(domain='xyz.com')
    image_cid = make_msgid()
    image_cid2 = make_msgid()
    # if `domain` argument isn't provided, it will 
    # use your computer's name

    # set an alternative html body
    msg.add_alternative("""\
    <html>
        <body>
            <p>{date}<br><br>           
            </p>

            <p>{mail_content}<br><br>           
            </p>

            <p>La Jolla:<br>           
            </p>

            <img src="cid:{image_cid}"><br>

            <p>Spring House:<br>

            </p>        

            <img src="cid:{image_cid2}">
        </body>
    </html>
    """.format(mail_content = mail_content, date = today, image_cid=image_cid[1:-1], image_cid2=image_cid2[1:-1]), subtype='html')
    # image_cid looks like <long.random.number@xyz.com>
    # to use it as the img src, we don't need `<` or `>`
    # so we use [1:-1] to strip them off


    png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\SH_LCMS_Status_2020_08_07.png'

    png_link = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\LJ_LCMS_Status_2020_08_07.png'

    # now open the image and attach it to the email
    with open(png_link, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid)


    # now open the image and attach it to the email
    with open(png_link2, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2)


    #Create SMTP session for sending the mail
    try:
        server = smtplib.SMTP('smtp.na.jnj.com', 25)
        server.ehlo()
        text = msg.as_string()
        server.sendmail(mail_sender, mail_receiver, text)
        server.close()
        print(f'\nEmail sent! \n{mail_subject}')
    except:
        print('Something went wrong...')
        pass

report_lcms_status(instr_status = 1)
report_lcms_status(instr_status = 0)


Email sent! 
OMG Super Important Message: failed!

Email sent! 
Great, QC passed for all instruments!


## Define a function with two status: LJ and SH

In [3]:
def report_lcms_status(LJ_instr_status, SH_instr_status):

    today = datetime.datetime.today().strftime("%B %m, %Y, %H:%M:%S")
    # strftime('%Y_%m_%d_%Hh%mm')

    msg = EmailMessage()

    mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
    mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_LJ = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_SH = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com']

    
    mail_receiver_list_US = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                             'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    
    if (LJ_instr_status == 0) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list1
        mail_subject = 'Great, QC passed for all instruments!'
        mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

    elif (LJ_instr_status == 0) & (SH_instr_status == 1):
        mail_receiver = mail_receiver_list_SH
        mail_subject = 'LJ: Passed; SH: Failed!'
        mail_content = f'Hey there, \n\nQC passed for LJ but not SH. Please doulbe check Spring House instruments!'        

    elif (LJ_instr_status == 1) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list_LJ
        mail_subject = 'LJ: Failed; SH: Passed!'
        mail_content = f'Hey there, \n\nQC passed for SH but not LJ. Please doulbe check La Jolla instruments!'   
        
    else:
        mail_receiver = mail_receiver_list_US
        mail_subject = 'OMG Super Important Message: QC failed for both LJ and SH!'
        mail_content = f'Hey there, \n\nQC failed. Please doulbe check both Spring House and La Jolla instruments!'


    mail_receiver = ', '.join(mail_receiver)


    # generic email headers
    msg['Subject'] = mail_subject

    msg['From'] = mail_sender
    msg['To'] = mail_receiver

    # set the plain text body
    msg.set_content('This is a plain text body.')

    # now create a Content-ID for the image
    # image_cid = make_msgid(domain='xyz.com')
    image_cid = make_msgid()
    image_cid2 = make_msgid()
    # if `domain` argument isn't provided, it will 
    # use your computer's name

    # set an alternative html body
    msg.add_alternative("""\
    <html>
        <body>
            <p>{date}<br><br>           
            </p>

            <p>{mail_content}<br><br>           
            </p>

            <p>La Jolla:<br>           
            </p>

            <img src="cid:{image_cid}"><br>

            <p>Spring House:<br>

            </p>        

            <img src="cid:{image_cid2}">
        </body>
    </html>
    """.format(mail_content = mail_content, date = today, image_cid=image_cid[1:-1], image_cid2=image_cid2[1:-1]), subtype='html')
    # image_cid looks like <long.random.number@xyz.com>
    # to use it as the img src, we don't need `<` or `>`
    # so we use [1:-1] to strip them off


    png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_LCMS_Status_2020_08_07.png'

    png_link = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\LJ_LCMS_Status_2020_08_07.png'

    # now open the image and attach it to the email
    with open(png_link, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid)


    # now open the image and attach it to the email
    with open(png_link2, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2)

    #Create SMTP session for sending the mail
    try:
        server = smtplib.SMTP('smtp.na.jnj.com', 25)
        server.ehlo()
        text = msg.as_string()
        server.sendmail(mail_sender, mail_receiver, text)
        server.close()
        print(f'\nEmail sent! \n{mail_subject}')
    except:
        print('Something went wrong...')
        pass

    
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 0)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 0)


Email sent! 
OMG Super Important Message: QC failed for both LJ and SH!

Email sent! 
LJ: Failed; SH: Passed!

Email sent! 
LJ: Passed; SH: Failed!

Email sent! 
Great, QC passed for all instruments!


## Four figures

In [8]:
def report_lcms_status(LJ_instr_status, SH_instr_status):

    today = datetime.datetime.today().strftime("%B %m, %Y, %H:%M:%S")
    # strftime('%Y_%m_%d_%Hh%mm')

    msg = EmailMessage()

    mail_sender = '(US Analytical Chemistry)rzhou11@its.jnj.com'
    mail_receiver_list1 = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_LJ = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    mail_receiver_list_SH = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com']

    
    mail_receiver_list_US = ['rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                           'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com',\
                             'rzhou11@its.jnj.com', 'rzhou11@its.jnj.com']
    
    
    if (LJ_instr_status == 0) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list1
        mail_subject = 'Great, QC passed for all instruments!'
        mail_content = f'Hey there, \n\nQC passed, all intruments look fine!'

    elif (LJ_instr_status == 0) & (SH_instr_status == 1):
        mail_receiver = mail_receiver_list_SH
        mail_subject = 'LJ: Passed; SH: Failed!'
        mail_content = f'Hey there, \n\nQC passed for LJ but not SH. Please doulbe check Spring House instruments!'        

    elif (LJ_instr_status == 1) & (SH_instr_status == 0):
        mail_receiver = mail_receiver_list_LJ
        mail_subject = 'LJ: Failed; SH: Passed!'
        mail_content = f'Hey there, \n\nQC passed for SH but not LJ. Please doulbe check La Jolla instruments!'   
        
    else:
        mail_receiver = mail_receiver_list_US
        mail_subject = 'OMG Super Important Message: QC failed for both LJ and SH!'
        mail_content = f'Hey there, \n\nQC failed. Please doulbe check both Spring House and La Jolla instruments!'


    mail_receiver = ', '.join(mail_receiver)


    # generic email headers
    msg['Subject'] = mail_subject

    msg['From'] = mail_sender
    msg['To'] = mail_receiver

    # set the plain text body
    msg.set_content('This is a plain text body.')

    # now create a Content-ID for the image
    # image_cid = make_msgid(domain='xyz.com')
    image_cid1 = make_msgid()
    image_cid1b = make_msgid()
    image_cid2 = make_msgid()
    image_cid2b = make_msgid()
    # if `domain` argument isn't provided, it will 
    # use your computer's name

    # set an alternative html body
    msg.add_alternative("""\
    <html>
        <body>
            <p>{date}<br><br>           
            </p>

            <p>{mail_content}<br><br>           
            </p>

            <p>La Jolla:<br>           
            </p>

            <img src="cid:{image_cid1}"><br>
            <img src="cid:{image_cid1b}"><br>

            <p>Spring House:<br>

            </p>        

            <img src="cid:{image_cid2}">
            <img src="cid:{image_cid2b}">
        </body>
    </html>
    """.format(mail_content = mail_content, date = today,\
               image_cid1=image_cid1[1:-1], image_cid1b=image_cid1b[1:-1],\
               image_cid2=image_cid2[1:-1], image_cid2b=image_cid2b[1:-1]), subtype='html')
    # image_cid looks like <long.random.number@xyz.com>
    # to use it as the img src, we don't need `<` or `>`
    # so we use [1:-1] to strip them off


    png_link2 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_LCMS_Status_2020_08_08.png'
    png_link2b = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_Legend.png'

    png_link1 = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\LJ_LCMS_Status_2020_08_08.png'
    png_link1b = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\LJ_Legend.png'

    # now open the image and attach it to the email
    with open(png_link1, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid1)

    with open(png_link1b, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid1b)        
        
        

    # now open the image and attach it to the email
    with open(png_link2, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2)

    with open(png_link2b, 'rb') as img:

        # know the Content-Type of the image
        maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

        # attach it
        msg.get_payload()[1].add_related(img.read(), 
                                             maintype=maintype, 
                                             subtype=subtype, 
                                             cid=image_cid2b)                
        
        
    #Create SMTP session for sending the mail
    try:
        server = smtplib.SMTP('smtp.na.jnj.com', 25)
        server.ehlo()
        text = msg.as_string()
        server.sendmail(mail_sender, mail_receiver, text)
        server.close()
        print(f'\nEmail sent! \n{mail_subject}')
    except:
        print('Something went wrong...')
        pass

    
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 1, SH_instr_status = 0)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 1)
report_lcms_status(LJ_instr_status = 0, SH_instr_status = 0)


Email sent! 
OMG Super Important Message: QC failed for both LJ and SH!

Email sent! 
LJ: Failed; SH: Passed!

Email sent! 
LJ: Passed; SH: Failed!

Email sent! 
Great, QC passed for all instruments!


In [6]:
png_link2b = r'\\rndusshnas02\ds_disc_chem\AnalyticalChem-SH\LCMS\qc_check\Report\Figure\SH_Legend.png'
os.startfile(png_link2b)